## "The mother ship" 
This notebook organizes inpts and function calls for creating the monte carlo csv file

##### Import functions neccessary for calling scripts

In [1]:
# import monte-carlo functions
from get_date import get_date
from get_lat_lon_indices import get_lat_lon_indices
from get_vessel_type import get_vessel_type
from get_vte_probability import get_vte_probability
from get_oil_capacity import get_oil_capacity
from get_length_origin_destination import get_length_origin_destination
import time

# import python functions needed for input variables
from datetime import datetime, date, timedelta
import arrow

##### These are the input variables needed to run the functions for the monte-carlo setup

In [2]:
# spill date
start_date = datetime(2015, 1, 1, 0, 30)
end_date   = datetime(2018, 12, 31, 23, 55)
delta_time = timedelta(hours = 1)

# spill location
ais_data_year = 2018
n_locations = 1  #desired number of latitude and longitude values
geotiff_directory = '/Users/rmueller/Data/MIDOSS/AIS/ShipTrackDensityGeoTIFFs/'
upsample_factor = 1 # GeoTIFF lat/lon will be sampled at 1/upsample_factor [km] resolution
vessel_types      = ["tanker", "atb", "barge", "cargo", "cruise",
                     "ferry", "fishing", "smallpass", "other"]

##### Imports and definitions for testing scripts and validating output

In [3]:
# these aren't used or needed for function calls
from rasterio.enums import Resampling
import rasterio as rio

#### Get the spill date

In [4]:
# get vessel traffic exposure probability by month
vte_probability = get_vte_probability( geotiff_directory )

spill_date = get_date( start_date, 
             end_date, 
             vte_probability,
             delta_time
            )
spill_month = spill_date.month
spill_month_name  = arrow.get(spill_date).format('MMMM') # string
f'{spill_month:02.0f}'

'06'

####  Get spill lat/lon value(s) as well as index values for extracting vessel time exposure values from GeoTIFFS

In [5]:
[spill_lat, spill_lon, x_index, y_index, data_val] = get_lat_lon_indices(geotiff_directory, spill_month, n_locations, upsample_factor)    

Randomly selecting spill location from all-traffic GeoTIFF:
...Creating 2D array of lat/lon strings (this may take a moment)
...Selecting 1 location(s)
...Creating vectors of lat/lon pairs and x-index/y-index pairs
That's a wrap, folks!


In [6]:
spill_lat

array([-123.256])

In [7]:
spill_lon

array([49.25253])

In [8]:
x_index

array([211])

In [9]:
y_index

array([484])

In [10]:
data_val

array([114.09422302])

In [11]:
# load geotiff and access "data_val" with [x_index, y_index] to verify that the output is equivalent

traffic_reader = rio.open(f'{geotiff_directory}all_2018_{spill_month:02.0f}.tif')

# dataset closes automatically using the method below

with traffic_reader as dataset:
    # resample data to target shape
    data = dataset.read(1,
        out_shape=(
            dataset.count,
            int(dataset.height * upsample_factor),
            int(dataset.width * upsample_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

print(f'compare output data_val = {data_val} hours/km^2 with {data[x_index,y_index]} hours/km^2')    

compare output data_val = [114.09422302] hours/km^2 with [114.09422] hours/km^2


####  Get vessel type 

In [12]:
vessel_type = get_vessel_type(geotiff_directory, 
                              vessel_types, 
                              ais_data_year, 
                              n_locations, 
                              spill_month,  # from get_date
                              x_index,      # from get_lat_lon_indices
                              y_index       # from get_lat_lon_indices
                              )
vessel_type

[array(['barge'], dtype='<U9')]

In [13]:
# this is just a test of the get_oil_capacity script
for vessel in vessel_types:
    [fuel, tank] = get_oil_capacity(60, vessel)
    print(f'{vessel} [fuel, tank]: {fuel:4.2e} m^3, {tank:4.2e} m^3')



tanker [fuel, tank]: 7.35e+05 m^3, 1.81e+07 m^3
atb [fuel, tank]: 4.50e+06 m^3, 3.84e+06 m^3
barge [fuel, tank]: 4.50e+06 m^3, 3.84e+06 m^3
cargo [fuel, tank]: 7.35e+05 m^3, 0.00e+00 m^3
cruise [fuel, tank]: 4.01e+05 m^3, 0.00e+00 m^3
ferry [fuel, tank]: 4.84e+04 m^3, 0.00e+00 m^3
fishing [fuel, tank]: 3.23e+18 m^3, 0.00e+00 m^3
smallpass [fuel, tank]: 2.43e+03 m^3, 0.00e+00 m^3
other [fuel, tank]: 2.43e+03 m^3, 0.00e+00 m^3


#### get length origin destination
The shapefiles for this piece aren't yet done, so we need to use january and cargo

In [23]:
vessel_type = 'cargo'
month = '01'
spill_lon = -124.9243
spill_lat = 50.18442
search_radius = 0.5 #km
shapefile_directory = shapefile_directory  = f'/Users/rmueller/Data/MIDOSS/{vessel_type}_2018_{month}/'
tic = time.time()
[length, origin, destination] = get_length_origin_destination( 
    shapefile_directory, 
    vessel_type, 
    month, 
    spill_lat, 
    spill_lon, 
    search_radius 
)
toc = time.time()-tic

'this is going to take a minute....'

In [15]:
toc

45.281166791915894

In [16]:
from salishsea_tools import geo_tools

#### Determine whether spill is fuel or cargo and then determine fuel type
Note: I approach this by first determining if spill is fuel or cargo and then determining the type of fuel if it's cargo.  The caveat here is that some of the barge traffic will be determined to be fuel-spill risk only.  For the barge case only, I have set up two return variables for the get_oil_type script.  One variable is "fuel_flag".  If fuel_flag == 1 then the spill is a fuel_spill.  

The scripts for getting oil cargo type are called as follows:

- oil_type = get_oil_type_tanker(
                    master_dir, 
                    master_file, 
                    origin, 
                    destination,
                    random_generator
                )

- oil_type = get_oil_type_atb(
                    master_dir, 
                    master_file, 
                    origin, 
                    destination,
                    random_generator
                )

- [oil_type, fuel_flag] = get_oil_type_barge(
                    master_dir, 
                    master_file, 
                    origin, 
                    destination,
                    random_generator
                )


In [43]:
# first, I'm going to retro-actively add some variables that can be placed
# at the top of the run script
import yaml
import numpy
from get_oil_type_atb import get_oil_type_atb
from get_oil_type_tanker import get_oil_type_tanker
from get_oil_type_barge import get_oil_type_barge

master_dir  = '/Users/rmueller/Projects/MIDOSS/analysis-rachael/notebooks/monte_carlo/'
master_file = 'master.yaml'
fuel_file   = 'fuel_by_vessel.yaml'

with open(f'{master_dir}{master_file}') as file:
        master = yaml.safe_load(file)
        
with open(f'{master_dir}{fuel_file}') as file:
        fuel = yaml.safe_load(file)

tank_vessels = master['categories']['tank_vessels']

# initialize fuel_oil_type, which will be given a real value if spill
# is determined to be a fuel-spill. Currently, cargo_oil_type = [] for 
# non-cargo spills as well.  Do we want a non-null value for these?
fuel_oil_type = []


# ~~~  TESTING ~~~~
# This section is for testing purposes.  It's not part of code

random_seed = []
random_generator = numpy.random.default_rng(random_seed)

vessel_type = 'barge'

# ~~~  END TESTING ~~~~

In [45]:
# get cargo oil type.  I think these functions make great candidates for being
# put in and accessed by a dictionary!  That will be the next iteration.
vessel_type = 'barge'

if vessel_type in tank_vessels:
    
    # fuel-spill or cargo-spill?
    probability = [
        master['vessel_attributes'][vessel_type]['probability_cargo'],
        master['vessel_attributes'][vessel_type]['probability_fuel']
    ]
    
    # fuel_flag = 1 indicates a fuel-spill
    fuel_flag = random_generator.choice(
                    [0, 1], 
                    p = probability)
    
    if not fuel_flag:
        if vessel_type == 'atb':
            cargo_oil_type = get_oil_type_atb(
                  master_dir, 
                  master_file, 
                  origin, 
                  destination,
                  random_generator
              )
        elif vessel_type == 'barge':
            [cargo_oil_type, fuel_flag] = get_oil_type_barge(
                  master_dir, 
                  master_file, 
                  origin, 
                  destination,
                  random_generator
              )
        elif vessel_type == 'tanker':
            cargo_oil_type = get_oil_type_tanker(
                        master_dir, 
                        master_file, 
                        origin, 
                        destination,
                        random_generator
                    )
    # The indentation here is to catch cases both the cases that are originally 
    # determined as fuel spills as well as cargo spills converted to fuel-spills         
    if fuel_flag:
        fuel_oil_type = random_generator.choice(
                    ['bunker', 'diesel'], 
                    p = [
                        fuel[vessel_type]['bunker'], 
                        fuel[vessel_type]['diesel']
                    ]
        )
        
# fuel-type for non-tank traffic
else:
    fuel_oil_type = random_generator.choice(
                    ['bunker', 'diesel'], 
                    p = [
                        fuel[vessel_type]['bunker'], 
                        fuel[vessel_type]['diesel']
                    ]
        )

### Calculate spill volume